In [124]:
import pickle
from collections import defaultdict
import numpy as np
from operator import itemgetter
import pandas as pd

In [125]:
import spacy
nlp = spacy.load('en_core_web_lg')
stopwords = nlp.Defaults.stop_words

In [ ]:
import os
#os.chdir("structural_optimization")
#os.chdir("smo")
#os.chdir("anja3")
#os.chdir("driver_glance_behaviour")
os.chdir("cov")
lvl = 0.5

work_dict = pickle.load(open("work_dict_lvl{}.p".format(lvl), "rb"))
#work_dict = pickle.load(open("work_dict_test_lvl1.p", "rb"))

In [ ]:
from requestWorksV3 import *
import json

db_engine = create_engine('sqlite:///COV.sqllite', echo=False)
Session.configure(bind=db_engine)
db_session = Session()

work_dict = {}

for res in db_session.query(CrossRefWork).yield_per(1000):
    #print(work)
    work = json.loads(res.json_content)
    doi = res.doi
    
    work_dict[doi] = work

In [127]:
#work_dict = pickle.load(open("work_dict_lvl1.p", "rb"))
#work_dict = pickle.load(open("work_dict_lvl2.p", "rb"))
#work_dict = pickle.load(open("crashworthiness/work_dict_lvl3.p", "rb"))
len(work_dict)

292731

In [128]:
from gensim.models import Word2Vec, KeyedVectors
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [129]:
def get_reference_dois(work):
    result = []
    if "reference" in work:
        for ref in work["reference"]:
            if "DOI" in ref:
                result.append(ref["DOI"])
        #if "DOI" in work:
            #result.append(work["DOI"])
    return result

In [130]:
issn_counts = defaultdict(int)
issn2journalTitle = {}

for work in work_dict.values():
    if "ISSN" in work:
        for issn in work["ISSN"]:
            issn_counts[issn] += 1
            if not issn in issn2journalTitle:
                if "container-title" in work:
                    issn2journalTitle[issn] = ";".join(work["container-title"])

In [131]:

#pd.DataFrame(index=issn_counts.keys(), data=issn_counts)
#pd.DataFrame(index=issn_counts.keys(), data={count: issn for issn, count in issn_counts.items()})
df = pd.DataFrame(index=issn_counts.keys(), columns=["Titel", "Publikationen"])
for issn, count in sorted(issn_counts.items(), key=itemgetter(1), reverse=True):
    df.at[issn, "Titel"] = issn2journalTitle[issn]
    df.at[issn, "Publikationen"] = count

df.to_excel("journal_list.xlsx")

In [132]:
#jt = set()
#f = open("journal_list.csv", "w", encoding="utf-8")
#f.write("Count\tJournalTitle\tISSN\n")
#for issn, count in sorted(issn_counts.items(), key=itemgetter(1), reverse=True):
#    if not (count, issn2journalTitle[issn]) in jt:
#        print(count, issn, issn2journalTitle[issn])
#        jt.add((count, issn2journalTitle[issn]))
#        f.write("{}\t{}\t{}\n".format(count, issn2journalTitle[issn], issn))
#f.close()

In [133]:
references_list = []
ref_counts = defaultdict(int)
for work in work_dict.values():
    #print(work)
    work_refs = get_reference_dois(work)
    #print(work_refs)
    #break
    for doi in work_refs:
        ref_counts[doi] += 1
    work_refs.append(work["DOI"])
    if len(work_refs) > 2:
        references_list.append(work_refs)
len(references_list)

158370

In [134]:
ref_counts_by_year = defaultdict(list)
for doi, count in ref_counts.items():
    #work = work_dict[doi]
    try:        
        y = work_dict[doi]["issued"]["date-parts"][0][0]
        if y:
            ref_counts_by_year[y].append(count)
    except KeyError:
        continue

In [135]:
avg_ref_count = defaultdict(int)
median_ref_count = defaultdict(int)
var_ref_count = defaultdict(int)
n_ref_count = defaultdict(int)
lambda_mle_ref_count = defaultdict(int)
percentile_ref_count = defaultdict(int)
for year, counts in ref_counts_by_year.items():
    if len(counts) > 20:
        avg_ref_count[year] = np.average(counts)
        median_ref_count[year] = np.median(counts)
        var_ref_count[year] = np.var(counts)
        n_ref_count[year] = len(counts)
        lambda_mle_ref_count[year] = len(counts) / sum(counts)
        percentile_ref_count[year] = - np.log(0.1) / lambda_mle_ref_count[year]
        
#np.average(ref_counts_by_year[2017])

sorted(avg_ref_count.items(), reverse=True)
sorted(median_ref_count.items(), reverse=True)
sorted(var_ref_count.items(), reverse=True)

lambda_mle_ref_count_default = np.median([l for y, l in lambda_mle_ref_count.items() if y < 2000])

x = sorted(zip(avg_ref_count.keys(), avg_ref_count.values(), median_ref_count.values(), var_ref_count.values(), n_ref_count.values()), reverse=True)
#pprint(x)
x = sorted(zip(avg_ref_count.keys(), avg_ref_count.values(), n_ref_count.values(), percentile_ref_count.values(), lambda_mle_ref_count.values()), reverse=True)
#print(x)
x, lambda_mle_ref_count_default


#lambda_mle_ref_count_default

([(2020, 2.72962962962963, 540, 6.285204494580043, 0.36635006784260515),
  (2019, 1.3628203272042856, 6907, 3.138009769849855, 0.7337724423669394),
  (2018, 1.7233879222108495, 11724, 3.9682473391286837, 0.5802524127691165),
  (2017, 2.100274254370929, 11668, 4.8360601893136845, 0.4761282951114013),
  (2016, 2.450482879719052, 11390, 5.642445349478209, 0.4080828347246605),
  (2015, 2.8615443498229833, 10451, 6.588949362843739, 0.3494616464923427),
  (2014, 3.2079062957540265, 9562, 7.386477216324969, 0.3117298037425833),
  (2013, 3.3266011448320554, 9259, 7.659782206427216, 0.30060712314535243),
  (2012, 3.5453888755116783, 8306, 8.16355957362011, 0.28205650638413476),
  (2011, 3.6874177415109237, 7598, 8.490593123244825, 0.27119249027376235),
  (2010, 3.9809588116137746, 7405, 9.166496415445168, 0.25119576647783165),
  (2009, 4.141344778254649, 6990, 9.535798751357888, 0.24146745889180599),
  (2008, 4.189863726841219, 6531, 9.647517759101065, 0.23867124689372898),
  (2007, 5.257680250

In [136]:
ref_count_normalized = defaultdict(float)

for doi, count in ref_counts.items():
    try:
        year = work_dict[doi]["issued"]["date-parts"][0][0]
        if year in lambda_mle_ref_count:
            if year >= 2000:
                l = lambda_mle_ref_count[year]
            else:
                l = lambda_mle_ref_count_default
        else:
            l = lambda_mle_ref_count_default
        if l < lambda_mle_ref_count_default:
            l = lambda_mle_ref_count_default
        ref_count_normalized[doi] = l*count
    except KeyError:
        continue

len(ref_counts), len(ref_count_normalized)

(3114814, 139496)

In [137]:
#model = Word2Vec(iter=1, size=300, window=1000, min_count=5, workers=4, sg=1)  # an empty model, no training yet
#model.build_vocab(references_list)
#model.train(references_list)

load_model = True

fname = "vectors_db.rv"
#fname = "testRVnew.rv"


#load_model = True

#fname = "vectors_db.rv"

if not load_model or not fname in os.listdir():
    model = Word2Vec(references_list, size=300, window=1000, min_count=3, workers=16, sg=1, iter=20)
    
    reference_vectors = model.wv
    reference_vectors.save(fname)
    del model
else:
    reference_vectors = KeyedVectors.load(fname)

2020-03-18 16:13:18,262 : INFO : loading Word2VecKeyedVectors object from vectors_db.rv
2020-03-18 16:13:18,481 : INFO : loading vectors from vectors_db.rv.vectors.npy with mmap=None
2020-03-18 16:13:18,965 : INFO : setting ignored attribute vectors_norm to None
2020-03-18 16:13:18,965 : INFO : loaded vectors_db.rv


In [138]:
len(reference_vectors.vocab)

164535

In [139]:
#reference_vectors = model.wv
#del model

In [140]:
#fname = "vectors.rv"
#reference_vectors.save(fname)

In [141]:
import numpy as np

In [142]:
X_dict = {}

for doi in reference_vectors.vocab.keys():
    newX = {}
    newX["lvl"] = 0
    newX["vec"] = reference_vectors[doi]
    X_dict[doi] = newX
print("Size of X_dict:", len(X_dict))
if 0:
    for lvl in range(1,2):
        print ("Level:", lvl)
        dois = set(work_dict.keys() - X_dict.keys())
        print("No. DOIs:", len(dois), "/", len(work_dict))
        for doi in dois:
            work = work_dict[doi]
        #for doi, work in work_dict.items():
            if not doi in X_dict:
                ref_dois = get_reference_dois(work)
                ref_dois = set(ref_dois) & X_dict.keys()            
                if len(ref_dois) >= 3:
                    vec = np.zeros(300)
                    for ref_doi in ref_dois:
                        #print(work_dict[ref_doi])
                        vec = vec + X_dict[ref_doi]["vec"]
                    vec = vec / len(ref_dois)
                    newX = {}
                    newX["lvl"] = lvl
                    newX["vec"] = vec
                    X_dict[doi] = newX
                pass
        print("Size of X_dict:", len(X_dict))

    
    work_dict0 = pickle.load(open("work_dict_lvl0.p", "rb"))
    len(X_dict), len(work_dict0), len(work_dict), len(X_dict.keys() & work_dict.keys()), len(X_dict.keys() & work_dict0.keys()), len(reference_vectors.vocab.keys() & work_dict0.keys())

Size of X_dict: 164535


In [143]:
from sklearn.manifold import TSNE

In [144]:
len(reference_vectors.vocab.keys())

164535

In [177]:
dois_to_plot = list(reference_vectors.vocab.keys() & work_dict.keys())
#dois_to_plot = list(reference_vectors.vocab.keys())
print(len(dois_to_plot), "possible")

def doi_plot_filter(doi):
    if not "title" in work_dict[doi]:
        return False
    if len(work_dict[doi]["title"]) == 0:
        return False
    if not work_dict[doi]["issued"]["date-parts"][0][0]:
        return False
    if work_dict[doi]["issued"]["date-parts"][0][0] >= 2014:
        return True
    elif ref_counts[doi] >= 20:
        return True
    elif work_dict[doi]["issued"]["date-parts"][0][0] >= 2008 and ref_counts[doi] >= 10:
        return True
    elif work_dict[doi]["issued"]["date-parts"][0][0] >= 2012 and ref_counts[doi] >= 5:
        return True
    else:
        return False

test_dois = [
    "10.1065/lca2008.02.376",
    "10.1016/j.cirp.2010.05.004",
    #"10.1007/s11367-014-0788-0",
    "10.1007/978-3-642-01421-5",
    "10.1108/09564230310474138",
    "10.1080/17517575.2012.683812",
    "10.1002/bse.414",
    "10.1111/j.1468-2370.2007.00202.x"
]

test_dois = [
    "10.1371/journal.pone.0031092",
    "10.1093/nar/gks017",
    "10.1016/j.chembiol.2012.01.015",
    "10.1111/j.1462-2920.2012.02719.x",
    "10.1021/ja3072397",
    "10.1371/journal.ppat.1002760",
    "10.1111/1462-2920.12024",
    "10.1371/journal.ppat.1004744",
    "10.1186/1475-2859-13-88",
    "10.1016/j.mimet.2011.11.015",
    "10.1099/jmm.0.05069-0"
]

test_dois = [
    "10.1007/s001580050176", # A 99 line topology optimization code written in Matlab
    "10.1038/451652a" # Building better batteries
]

test_dois = [
    "10.1038/nature14539", # Deep learning
    "10.1038/nature16961",
    "10.1038/nature21056"
]

test_dois = [
    "10.1016/j.tcs.2014.11.028",
    "10.1007/978-3-662-05094-1",
    "10.1038/nature14422",
    "10.1126/science.1165893",
    "10.1109/4235.996017",
    "10.1007/978-3-642-17339-4"
]


test_dois = [
    "10.1016/j.addma.2016.06.010", # Topology optimization of 3D self-supporting structures for additive manufacturing
    "10.1007/s00158-015-1261-9" #Topology and shape optimization methods using evolutionary algorithms: a review
]

#test_dois = [
#    "10.1016/j.actamat.2016.07.019" # Additive manufacturing of metals
#]

#test_dois = [
#    "10.1115/1.4004465", # Multidisciplinary Design Optimization for Complex Engineered Systems: Report From a National Science Foundation Workshop
#    "10.1007/s00158-012-0763-y", # Extensions to the design structure matrix for the description of multidisciplinary design, analysis, and optimization processes
#    "10.1016/j.paerosci.2011.05.001"
#    
#]

test_dois = [
    "10.1016/j.cad.2015.04.001",
]

test_dois = [
    "10.1561/1500000011",
    "10.1145/1014052.1014073",
]

test_dois = [
    "10.1016/j.trf.2014.06.016"   # Effects of adaptive cruise control and highly automated driving on workload and situation awareness: A review of the empirical evidence
]

test_dois = [
    "10.1126/sciadv.1602614", #Data-driven discovery of partial differential equations
    #"10.1146/annurev-fluid-010816-060042",
    "10.1063/1.4927765",
    "10.1017/jfm.2016.615"
]

test_dois = [
    "10.1038/s41586-020-2012-7" #cov
]

for doi in test_dois:
    if doi in work_dict:
        print(work_dict[doi]["title"])

def doi_plot_filter2(doi, min_sim=0.35):
    
    
    #min_sim = 0.35
    
    for test_doi in test_dois:
        if reference_vectors.similarity(doi, test_doi) > min_sim:
            return True
    return False


#dois_to_plot = [doi for doi, count in sorted(ref_count_normalized.items(), key=itemgetter(1), reverse=True)][:3000]
        
#dois_to_plot = [doi for doi in dois_to_plot if doi_plot_filter(doi)]
dois_to_plot = [doi for doi in dois_to_plot if doi_plot_filter2(doi, 0.485)]

dois_to_plot = [doi for doi in dois_to_plot if "title" in work_dict[doi]]
dois_to_plot = [doi for doi in dois_to_plot if len(work_dict[doi]["title"]) > 0]
dois_to_plot = [doi for doi in dois_to_plot if work_dict[doi]["issued"]["date-parts"][0][0]]
dois_to_plot = [doi for doi in dois_to_plot if work_dict[doi]["issued"]["date-parts"][0][0] >= 2008]
#dois_to_plot = [doi for doi in dois_to_plot if ref_counts[doi] >= 4]

#filter_issn = "2071-1050" # Sustainability
#filter_issn = "1615-1488" # SMO
#filter_issn = "0268-3768"
#dois_to_plot = [doi for doi in dois_to_plot if "ISSN" in work_dict[doi]]
#dois_to_plot = [doi for doi in dois_to_plot if filter_issn in work_dict[doi]["ISSN"]]

#test_dois = ["10.1016/j.actamat.2016.02.014"] # Laser powder-bed fusion additive manufacturing: Physics of complex melt flow and formation mechanisms of pores, spatter, and denudation zones

#dois_to_plot = [doi for doi in dois_to_plot if doi_plot_filter2(doi, 0.8)]

print(len(dois_to_plot), "selected")

dois_to_plot = [doi for doi in dois_to_plot if doi in ref_count_normalized]
#dois_to_plot = [doi for doi in dois_to_plot if ref_count_normalized[doi] >= 1.5]

#dois_to_plot = [doi for doi, count in sorted([doi, ref_count_normalized[doi] for doi in dois_to_plot], key=itemgetter(1), reverse=True)][:3000]

a = [(doi, ref_count_normalized[doi]) for doi in dois_to_plot]
b = sorted(a, key=itemgetter(1), reverse=True)[:2500]
dois_to_plot = [doi for doi, count in b]

print(len(dois_to_plot), "selected")


31261 possible
['A pneumonia outbreak associated with a new coronavirus of probable bat origin']
2303 selected
2303 selected


In [178]:
X = reference_vectors[dois_to_plot]

tsne = TSNE(n_components=2, metric="cosine")
X_embedded = tsne.fit_transform(X)
#X_embedded = TSNE(n_components=2).fit_transform(X)


In [179]:
if 0:
    dois_to_plot = list(X_dict.keys() & work_dict0.keys())
    print(len(dois_to_plot))
    X = np.zeros((len(dois_to_plot), 300))
    for i, doi in enumerate(dois_to_plot):
        X[i, :] = X_dict[doi]["vec"]

    X_embedded = TSNE(n_components=2).fit_transform(X)

In [180]:
C_embedded = TSNE(n_components=3, metric="cosine").fit_transform(X)
#C_embedded = TSNE(n_components=3).fit_transform(X)


In [181]:
#X_embedded *= 3.5

In [182]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
#colors_r = np.floor(MinMaxScaler().fit_transform(C_embedded[:,0])*256)

cmin = np.min(C_embedded)
cmax = np.max(C_embedded)
cmin, cmax

(-18.861486, 21.46803)

In [183]:
colors = (C_embedded - cmin) / (cmax - cmin) * 256
colors = np.floor(colors)
colors

array([[107.,  95.,   4.],
       [148., 126., 170.],
       [115.,  83.,   2.],
       ...,
       [ 64.,  42.,  94.],
       [104.,  46.,  25.],
       [100.,  69., 120.]], dtype=float32)

In [184]:
X_embedded[0,:]

array([-26.923471,  31.237288], dtype=float32)

In [185]:
import pandas as pd

In [186]:
columns = ["doi", "title", "year", "link", "authors", "containerTitles", "refCount", "xval", "yval", "rgb", "clusterNo"]
df = pd.DataFrame(index=dois_to_plot, columns=columns)

In [187]:
#from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
#
#max_d = 0.2
#
#Z = linkage(X, method='average', metric='cosine')
##Z = linkage(X, method='complete', metric='cosine')
#clLabels = fcluster(Z, max_d, criterion='distance') - 1
##clLabels = fcluster(Z, 50, criterion='maxclust') - 1
#
#cluster_ids = sorted(np.unique(clLabels))

In [188]:
#from sklearn.cluster import DBSCAN
#
#clLabels = DBSCAN(eps=0.1, min_samples=5, metric="cosine").fit_predict(X)
#cluster_ids = sorted(np.unique(clLabels))

In [189]:
int(np.sqrt(len(dois_to_plot)))

47

In [190]:
from sklearn.mixture import GaussianMixture

n = int(np.sqrt(len(dois_to_plot)))
print(n)

gmm = GaussianMixture(n_components=n, verbose=True)

clLabels = gmm.fit_predict(X)
cluster_ids = sorted(np.unique(clLabels))

47
Initialization 0
Initialization converged: True


In [191]:
[(cl, np.sum(clLabels==cl)) for cl in cluster_ids]

[(0, 73),
 (1, 46),
 (2, 104),
 (3, 51),
 (4, 18),
 (5, 21),
 (6, 87),
 (7, 132),
 (8, 19),
 (9, 88),
 (10, 22),
 (11, 101),
 (12, 31),
 (13, 18),
 (14, 48),
 (15, 37),
 (16, 39),
 (17, 65),
 (18, 46),
 (19, 46),
 (20, 92),
 (21, 53),
 (22, 10),
 (23, 86),
 (24, 61),
 (25, 55),
 (26, 62),
 (27, 54),
 (28, 15),
 (29, 61),
 (30, 16),
 (31, 25),
 (32, 18),
 (33, 28),
 (34, 17),
 (35, 61),
 (36, 44),
 (37, 63),
 (38, 45),
 (39, 81),
 (40, 47),
 (41, 7),
 (42, 17),
 (43, 70),
 (44, 41),
 (45, 72),
 (46, 10)]

In [192]:
clLabels, np.max(cluster_ids)

(array([35,  9, 35, ..., 43, 43,  6], dtype=int64), 46)

In [193]:
from gensim.models.phrases import *
from pprint import pprint
import operator

def phrase_lines(texts_temp):

    phrases = Phrases(texts_temp, min_count=10, threshold=10)
    bigram = Phraser(phrases)
    texts_phrased = [bigram[t] for t in texts_temp]

    phrase_scores = {}
    for phrase, score in phrases.export_phrases(texts_temp):
        phrase_scores[phrase] = score
    print("Bigramme")
    pprint(sorted(phrase_scores.items(), key=operator.itemgetter(1), reverse=True)[:50])

    texts_temp = texts_phrased
    phrases = Phrases(texts_temp, min_count=5, threshold=7)
    trigram = Phraser(phrases)
    texts_phrased = [trigram[t] for t in texts_temp]

    phrase_scores = {}
    for phrase, score in phrases.export_phrases(texts_temp):
        phrase_scores[phrase] = score
    print("Trigramme")
    pprint(sorted(phrase_scores.items(), key=operator.itemgetter(1), reverse=True)[:50])

    return texts_phrased, bigram, trigram

In [194]:
from gensim import corpora, models, similarities

titles = []
for doi in dois_to_plot:
    pub = work_dict[doi]
    if "title" in pub:
        if len(pub["title"]) > 0:
            pub_title = [s for s in pub["title"][0].lower().split() if not s in stopwords]
            titles.append(pub_title)

phrased_lines, bigram, trigram = phrase_lines(titles)

frequency = defaultdict(int)

for line in phrased_lines:
    for term in line:
        frequency[term] += 1

dictionary = corpora.Dictionary(phrased_lines)

corpus = [dictionary.doc2bow(line) for line in phrased_lines]

tfidf = models.TfidfModel(corpus, normalize=True) # step 1 -- initialize a model

2020-03-18 16:16:24,342 : INFO : collecting all words and their counts
2020-03-18 16:16:24,342 : INFO : PROGRESS: at sentence #0, processed 0 words and 0 word types
2020-03-18 16:16:24,373 : INFO : collected 21469 word types from a corpus of 20026 words (unigram + bigrams) and 2303 sentences
2020-03-18 16:16:24,373 : INFO : using 21469 counts as vocab in Phrases<0 vocab, min_count=10, threshold=10, max_vocab_size=40000000>
2020-03-18 16:16:24,373 : INFO : source_vocab length 21469
2020-03-18 16:16:24,514 : INFO : Phraser built with 43 phrasegrams
2020-03-18 16:16:24,623 : INFO : collecting all words and their counts
2020-03-18 16:16:24,623 : INFO : PROGRESS: at sentence #0, processed 0 words and 0 word types
2020-03-18 16:16:24,654 : INFO : collected 21614 word types from a corpus of 19246 words (unigram + bigrams) and 2303 sentences
2020-03-18 16:16:24,654 : INFO : using 21614 counts as vocab in Phrases<0 vocab, min_count=5, threshold=7, max_vocab_size=40000000>
2020-03-18 16:16:24,65

Bigramme
[(b'body area', 207.42995169082124),
 (b'middle east', 195.38673245614035),
 (b'severe acute', 182.16121212121215),
 (b'hepatitis c', 170.38888888888889),
 (b'respiratory syndrome', 128.161053160739),
 (b'east respiratory', 126.92559480894016),
 (b'receptor-binding domain', 120.27450980392157),
 (b'routing protocol', 108.33809924306138),
 (b'indoor positioning', 103.47310323383086),
 (b'sensor networks', 102.5947237496727),
 (b'wireless sensor', 95.78155418559378),
 (b'kalman filter', 85.19444444444444),
 (b'underwater acoustic', 84.33389450056117),
 (b'2019 novel', 78.64102564102564),
 (b'genetic programming', 66.46749226006192),
 (b'acute lymphoblastic', 66.05846153846154),
 (b'acute respiratory', 61.76013698630137),
 (b'spike protein', 57.836745689655174),
 (b'public sector', 45.87393162393162),
 (b'systematic review', 43.37171717171717),
 (b'acoustic sensor', 35.60364842454394),
 (b'crystal structure', 33.13117283950617),
 (b'indoor localization', 32.72713414634146),
 (b's

2020-03-18 16:16:24,904 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-03-18 16:16:24,935 : INFO : built Dictionary(6964 unique tokens: ['data', 'flexible', 'illumina', 'sequence', 'trimmer']...) from 2303 documents (total 18687 corpus positions)
2020-03-18 16:16:24,951 : INFO : collecting document frequencies
2020-03-18 16:16:24,951 : INFO : PROGRESS: processing document #0
2020-03-18 16:16:24,967 : INFO : calculating IDF weights for 2303 documents and 6963 features (18517 matrix non-zeros)


Trigramme
[(b'de novo', 982.4545454545455),
 (b'wireless_body area', 720.4666666666667),
 (b'escherichia coli', 686.1587301587301),
 (b'saudi arabia', 635.7058823529412),
 (b'critically ill', 600.3888888888888),
 (b'acute_lymphoblastic leukemia', 540.35),
 (b'hereditary angioedema', 514.6190476190476),
 (b'carboxylesterase 1', 415.6538461538462),
 (b'novel_coronavirus (2019-ncov)', 392.98181818181826),
 (b'monoclonal antibody', 377.8671328671329),
 (b'mhc class', 350.8766233766234),
 (b'rhesus macaques', 343.07936507936506),
 (b'ion channel', 308.7714285714286),
 (b'received signal', 280.7012987012987),
 (b'underwater_wireless sensor_networks', 267.66563467492256),
 (b'hepatitis e', 240.15555555555554),
 (b'underwater_acoustic sensor_networks', 224.36678200692043),
 (b'structural basis', 186.32758620689654),
 (b'middle_east respiratory_syndrome', 181.01054590570718),
 (b'wireless_sensor networks', 178.20053191489362),
 (b'routing protocols', 157.19272727272727),
 (b'health care', 140.3

In [195]:
clSizes = []
clTitles = defaultdict(list)

for cl in cluster_ids:    
    print("Cluster No.", cl)
    cl_dois = [doi for i, doi in enumerate(dois_to_plot) if clLabels[i] == cl]
    print("Size:", len(cl_dois))
    for doi in cl_dois:
        #pprint(work_dict[doi]["title"])
        pub = work_dict[doi]
        if "title" in pub:
            if len(pub["title"]) > 0:
                pub_title = [s for s in pub["title"][0].lower().split() if not s in stopwords]
                clTitles[cl].append(pub_title)        
    print()
    clSizes.append(len(cl_dois))

Cluster No. 0
Size: 73

Cluster No. 1
Size: 46

Cluster No. 2
Size: 104

Cluster No. 3
Size: 51

Cluster No. 4
Size: 18

Cluster No. 5
Size: 21

Cluster No. 6
Size: 87

Cluster No. 7
Size: 132

Cluster No. 8
Size: 19

Cluster No. 9
Size: 88

Cluster No. 10
Size: 22

Cluster No. 11
Size: 101

Cluster No. 12
Size: 31

Cluster No. 13
Size: 18

Cluster No. 14
Size: 48

Cluster No. 15
Size: 37

Cluster No. 16
Size: 39

Cluster No. 17
Size: 65

Cluster No. 18
Size: 46

Cluster No. 19
Size: 46

Cluster No. 20
Size: 92

Cluster No. 21
Size: 53

Cluster No. 22
Size: 10

Cluster No. 23
Size: 86

Cluster No. 24
Size: 61

Cluster No. 25
Size: 55

Cluster No. 26
Size: 62

Cluster No. 27
Size: 54

Cluster No. 28
Size: 15

Cluster No. 29
Size: 61

Cluster No. 30
Size: 16

Cluster No. 31
Size: 25

Cluster No. 32
Size: 18

Cluster No. 33
Size: 28

Cluster No. 34
Size: 17

Cluster No. 35
Size: 61

Cluster No. 36
Size: 44

Cluster No. 37
Size: 63

Cluster No. 38
Size: 45

Cluster No. 39
Size: 81

Cluster

In [196]:
clDict = {}

for cl in cluster_ids[:]:
    #cl_titles_phrased = []
    clTokens = []
    if len(clTitles[cl]) < 3:
        continue
    for title in clTitles[cl]:
        title_phrased = trigram[bigram[title]]
        #pprint(title_phrased)
        #cl_titles_phrased.append(title_phrased)
        for t in title_phrased:
            clTokens.append(t)
    #print(clTokens)
    
    cl_dois = [doi for i, doi in enumerate(dois_to_plot) if clLabels[i] == cl]
    
    clBow = dictionary.doc2bow(clTokens)
    clFrequency = {}
    for i, freq in clBow:
        clFrequency[i] = freq
    #print(clBow)
    vec = sorted(tfidf[clBow], key=itemgetter(1), reverse=True)
    
    print("Cluster No.", cl)
    print("Size:", len(cl_dois))
    print("Terms:", ", ".join(["{}".format(dictionary[i]) for i, v in vec[:20] if v >= 100 and clFrequency[i] >= 5]))
    print(", ".join(["{} ({}, {}, {:.2f})".format(dictionary[i], frequency[dictionary[i]], clFrequency[i], v) for i, v in vec[:20]]))
    #print([(dictionary[i], frequency[dictionary[i]], clFrequency[i], v) for i, v in vec])
    
    author_counts = defaultdict(int)
    for doi in cl_dois:
        work = work_dict[doi]
        if "author" in work:
            for author in work["author"]:
                if "family" in author:
                    author_counts[author["family"]] += 1
    top_authors = sorted(author_counts.items(), key=itemgetter(1), reverse = True)
    top_authors = [(a, n) for a,n in top_authors if n >= 2][:10]
    print("top_authors:", top_authors)
    
    print()
    
    
    clDict[cl] = {}
    clDict[cl]["id"] = cl
    clDict[cl]["size"] = len(cl_dois)
    clDict[cl]["dois"] = cl_dois
    clDict[cl]["vec"] =  [(dictionary[i], frequency[dictionary[i]], clFrequency[i], v) for i, v in vec]
    clDict[cl]["top_authors"] = top_authors
    

Cluster No. 0
Size: 73
Terms: 
wireless_sensor_networks (31, 25, 0.41), networks (36, 16, 0.25), routing (18, 13, 0.24), wireless_body_area (11, 11, 0.22), mobile (26, 12, 0.21), energy (27, 11, 0.19), scheme (19, 10, 0.18), mac (9, 8, 0.17), data (64, 12, 0.17), survey (32, 10, 0.16), protocol (13, 8, 0.16), sensor_networks (10, 7, 0.15), clustering (14, 7, 0.14), routing_protocol (16, 7, 0.13), wsns (10, 6, 0.12), routing_protocols (7, 5, 0.11), energy-efficient (10, 5, 0.10), gathering (4, 4, 0.10), based (75, 7, 0.09), secure (6, 4, 0.09)
top_authors: [('Liu', 21), ('Wang', 18), ('Kim', 7), ('Zhang', 7), ('Park', 6), ('Xu', 6), ('Zhao', 6), ('Gao', 5), ('Xiong', 5), ('Chen', 5)]

Cluster No. 1
Size: 46
Terms: 
porcine (7, 7, 0.28), coronavirus (126, 10, 0.20), protein (83, 7, 0.16), sars_coronavirus (24, 5, 0.16), protease (10, 4, 0.15), deltacoronavirus (3, 3, 0.14), pathogenesis (18, 4, 0.13), 3c-like (4, 3, 0.13), nucleocapsid (6, 3, 0.12), sars-cov (27, 4, 0.12), activity (27, 

Cluster No. 18
Size: 46
Terms: 
zika_virus (11, 8, 0.28), concentrates (5, 5, 0.20), platelet (6, 5, 0.19), dengue (7, 5, 0.19), reduction (4, 4, 0.16), light (14, 4, 0.13), ebola (13, 4, 0.13), detection (38, 5, 0.13), ultraviolet (3, 3, 0.13), viruses (22, 4, 0.12), virus (65, 5, 0.12), disease (33, 4, 0.11), plasma (9, 3, 0.11), pathogen (10, 3, 0.11), notifiable (2, 2, 0.10), acid (16, 3, 0.10), influenza_virus (17, 3, 0.10), humans (20, 3, 0.09), establishment (2, 2, 0.09), characterization, (2, 2, 0.09)
top_authors: [('Zhang', 7), ('Chen', 6), ('Seltsam', 4), ('Wang', 4), ('Holmes', 3), ('Lee', 3), ('Gravemann', 3), ('Huang', 3), ('Tang', 3), ('Müller', 3)]

Cluster No. 19
Size: 46
Terms: 
acute_lymphoblastic_leukemia (8, 8, 0.30), sequencing (24, 6, 0.18), variation (11, 5, 0.18), acute_lymphoblastic (4, 4, 0.17), leukaemia (4, 4, 0.17), mutations (20, 5, 0.16), schizophrenia (3, 3, 0.13), precursor (4, 3, 0.13), cytokine (4, 3, 0.13), data (64, 5, 0.12), genetic (26, 4, 0.12), 

Cluster No. 42
Size: 17
Terms: 
kinetochore (6, 6, 0.41), unconventional (2, 2, 0.16), eukaryotes (2, 2, 0.16), trypanosome (2, 2, 0.16), checkpoint (2, 2, 0.16), spindle (2, 2, 0.16), toolkit (3, 2, 0.15), brucei (3, 2, 0.15), kinetoplastid (3, 2, 0.15), cell (34, 3, 0.15), chromosome (5, 2, 0.14), evolutionary (17, 2, 0.11), dynamics (17, 2, 0.11), assembly (20, 2, 0.11), completely (1, 1, 0.09), hhpred (1, 1, 0.09), mpi (1, 1, 0.09), reimplemented (1, 1, 0.09), hmmer (1, 1, 0.09), 2019: (1, 1, 0.09)
top_authors: [('Akiyoshi', 5), ('Gull', 4), ('Hayashi', 2)]

Cluster No. 43
Size: 70
Terms: 
human (111, 14, 0.20), gene (39, 10, 0.20), htlv-1 (6, 6, 0.18), drosophila (6, 6, 0.17), expression (19, 7, 0.16), noncoding (6, 5, 0.14), ribosomal (6, 4, 0.12), cells (26, 5, 0.11), genome (63, 6, 0.10), rna (35, 5, 0.10), protein (83, 6, 0.10), chromatin (3, 3, 0.10), retinal (3, 3, 0.10), dystrophin (3, 3, 0.10), splicing (3, 3, 0.10), resistance (17, 4, 0.09), origin (19, 4, 0.09), intergen

In [197]:
from bokeh.colors import RGB

#year2alpha = lambda year: 1.5 / (1. + np.exp(0.2*(2018 - year))) + 0.25
year2alpha = lambda year: np.clip(2. / (1. + np.exp(0.10*(2018 - year))), a_min=0.1, a_max=None)


for i, doi in enumerate(dois_to_plot):
    pub = work_dict[doi]
    df.at[doi, "doi"] = doi
    df.at[doi, "title"] = pub["title"][0]
    if "container-title" in pub:
        df.at[doi, "containerTitle"] = ";".join(pub["container-title"])
    #line = pub["title"][0]
    #n = 8
    #df.at[doi, "title"] = "\n".join([line[j:j+n] for j in range(0, len(line), n)])
    df.at[doi, "issued"] = "-".join([str(x) for x in pub["issued"]["date-parts"][0]])
    df.at[doi, "refCount"] = ref_counts[doi]
    df.at[doi, "refCountNormalized"] = ref_count_normalized[doi]
    #df.at[doi, "scatterSize"] = 6.+ np.power(ref_counts[doi], 0.8)
    df.at[doi, "scatterSize"] = 4+2.5*np.power(ref_count_normalized[doi], 1.)
    df.at[doi, "xval"] = X_embedded[i,0]
    df.at[doi, "yval"] = X_embedded[i,1]
    df.at[doi, "rgb"] = RGB(colors[i,0], colors[i,1], colors[i,2])
    df.at[doi, "alpha"] = year2alpha(pub["issued"]["date-parts"][0][0])
    try:
        pub_authors = [a["family"] for a in pub["author"]]
        df.at[doi, "author"] = ", ".join(pub_authors)
    except KeyError:
        df.at[doi, "author"] = "unavailable"
    
    df.at[doi, "clusterNo"] = clLabels[i]
    

[year2alpha(y) for y in (2018, 2017, 2014, 2010, 2000, 1980)]

[1.0,
 0.95004162504212,
 0.802624679775096,
 0.6200510377447751,
 0.28370212980097553,
 0.1]

In [198]:
#df["scatterSize"] *= 0.75
df["scatterSize"] *= 1.5

In [199]:
df["xval"].max()
df["xval"].min()
x_span = df["xval"].max() - df["xval"].min()
y_span = df["yval"].max() - df["yval"].min()

In [200]:
np.zeros(10)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [201]:
#tsne.transform

In [202]:
##### clDict = pickle.load(open("clDict.p", "rb"))

label_source_df = pd.DataFrame(index=clDict.keys(), columns=["x", "y", "text"])

for cl_id, cluster in clDict.items():
    #print(cluster)
    cl_dois = cluster["dois"]
    x = np.zeros(len(cl_dois))
    y = np.zeros(len(cl_dois))
    w = np.zeros(len(cl_dois))
    r = np.zeros(len(cl_dois))
    g = np.zeros(len(cl_dois))
    b = np.zeros(len(cl_dois))
    n = 0
    
    
    for i, doi in enumerate(cl_dois):
        if doi in df.index:
            n += 1
            x[i] = df.at[doi, "xval"]
            y[i] = df.at[doi, "yval"]
            w[i] = df.at[doi, "refCountNormalized"]
        else:
            w[i] = 0
    #if n < 3:
    #    continue
    
    xx = np.median(x)
    yy = np.median(y)
    
    #xx = np.average(x, weights=w)
    #yy = np.average(y, weights=w)
    
    x_std = np.std(x/x_span)
    y_std = np.std(y/y_span)
    
    print()
    print(cl_id)
    print(", ".join([s for s, totalFreq, clFreq, v in cluster["vec"] if v>=0.1 and totalFreq>=3][:5]))
    print(xx,yy,len(cl_dois))
    print(x_std, y_std , np.sqrt(x_std**2 + y_std**2))
    print(np.sum(w))
    
    
    if np.sum(w) < 10:
        print("w too small")
        continue
    
    std_max = 0.1
    
    if x_std**2 + y_std**2 > std_max**2:
        print("std too big")
        #continue
    
    
    
    label_source_df.at[cl_id, "x"] = xx
    label_source_df.at[cl_id, "y"] = yy
    #label_source.at[cl_id, "text"] = ", ".join([s for s, n in cluster["top_authors"][:3]]) \
    #    + " | " + ",\n".join([s for s, totalFreq, clFreq, v in cluster["vec"] if v>=0.25 and clFreq>=3][:2])
    
    #label_source_df.at[cl_id, "text"] = ", ".join([s for s, totalFreq, clFreq, v in cluster["vec"] if v>=0.3])
    label_source_df.at[cl_id, "text"] = ", ".join([s for s, totalFreq, clFreq, v in cluster["vec"] if v>=0.1 and totalFreq>=3][:2])
    #label_source.at[cl_id, "text"] = ", ".join([s for s, totalFreq, clFreq, v in cluster["vec"] if v>=0.225 and clFreq>=2][:2])
    #label_source.at[cl_id, "text"] = ", ".join([s for s, totalFreq, clFreq, v in cluster["vec"] if v>=0.01 and clFreq>=1][:1])
    #label_source.at[cl_id, "text"] = ", ".join([s for s, totalFreq, clFreq, v in cluster["vec"]][:1])
    #label_source.at[cl_id, "text"] = ", ".join([s for s, totalFreq, clFreq, v in cluster["vec"] if v>=500])
    if len(label_source_df.at[cl_id, "text"]) == 0:
        label_source_df.at[cl_id, "text"] = np.nan
    
    top_authors = cluster["top_authors"]
    label_source_df.at[cl_id, "authors"] = ", ".join([a for a,n in top_authors][:3])
    
    #print(xx,yy,len(cl_dois),label_source_df.at[cl_id, "text"])
    

                 
    #xy = X_embedded[, dtype=int), :]
    #x = df["xval"].loc[cl_dois]
    #print()

#label_source = 
label_source_df = label_source_df[pd.notnull(label_source_df["x"])]
label_source_df = label_source_df[pd.notnull(label_source_df["text"])]

    


0
wireless_sensor_networks, networks, routing, wireless_body_area, mobile
40.25177001953125 -13.080853462219238 73
0.022296816984187142 0.025984917730798033 0.03423980135898394
230.57807060872415

1
porcine, coronavirus, protein, sars_coronavirus, protease
-1.8652465343475342 -10.985311508178711 46
0.050812303212273646 0.049699567775072825 0.07107698076568174
117.17963987859467

2
frankia, sp., isolated, ambrosia, production
-32.83207130432129 39.02933120727539 104
0.07741316339817451 0.07567859122433368 0.10825916606464443
256.3918230790151
std too big

3
middle_east_respiratory_syndrome, coronavirus, treatment, sars, convalescent
-9.132807731628418 -23.198823928833008 51
0.025852719300624456 0.04831347866287366 0.05479557751994981
277.4351149965737

4
toll-like, innate_immune, receptor, receptors, 8
-9.87115478515625 8.040507316589355 18
0.02849485626647302 0.04899496090338663 0.056678594086051054
36.80951512658132

5
carboxylesterase_1, hereditary_angioedema, gene, mutation, mutati

In [203]:
from bokeh.plotting import figure, output_file, show, save, output_notebook
from bokeh.models import HoverTool, Range1d, LabelSet, LassoSelectTool, CustomJS, TapTool, OpenURL
from bokeh.models.sources import ColumnDataSource
from bokeh.io import reset_output

reset_output()


#class hover(HoverTool):
#    hover.tooltips = [
#    ("index", "$index"),


hover = HoverTool()
hover.tooltips = [
    ("Title", "@title"),
    ("Autoren", "@author"),
    ("Veröffentlicht", "@issued"),
    ("Journal", "@containerTitle"),
    ("Referenzen", "@refCount"),
]




s1 = ColumnDataSource(data=df)


callback = CustomJS(code="""
// the event that triggered the callback is cb_obj:
// The event type determines the relevant attributes
console.log('Tap event occured at x-position: ' + cb_obj.x)
""")


def lasso_cb(x):
    print(x)

#lasso = LassoSelectTool(callback=lasso_cb)
lasso = LassoSelectTool()


p = figure(plot_width=1200, plot_height=1000, active_scroll = "wheel_zoom")
#p = figure(plot_width=1200, plot_height=1000, output_backend="webgl", active_scroll = "wheel_zoom")
p.circle(x='xval', y='yval', size="scatterSize", color="rgb", alpha="alpha", source=s1)

p.add_tools(hover)
p.add_tools(lasso)

p.js_on_event('tap', callback)

#p.toolbar.active_scroll = "wheel_zoom"

df["xval"].max()

left, right, bottom, top = -120, 120, -90, 90
left, right, bottom, top = -80, 80, -60, 60
left, right, bottom, top = -100, 100, -90, 90
p.x_range=Range1d(left, right)
p.y_range=Range1d(bottom, top)

try:
    label_source = ColumnDataSource(label_source_df)   

    p.scatter(x='x', y='y', source=label_source, size=8, color="black", marker="diamond")
    
    labels = LabelSet(x='x', y='y', x_offset=-20, y_offset=5, text='text', level='glyph', 
                  source=label_source, render_mode='canvas')
    #author_labels = LabelSet(x='x', y='y', x_offset=10, y_offset=-15, text='authors', level='glyph', 
    #              source=label_source, render_mode='canvas')

    p.add_layout(labels)
    #p.add_layout(author_labels)
except ValueError:
    pass
except NameError:
    pass

output_file("test.html")
#output_notebook()
#show(p)
#save(p)

from bokeh.layouts import widgetbox, column, row
from bokeh.models import ColumnDataSource
from bokeh.models.widgets import DataTable, DateFormatter, TableColumn

s2 = ColumnDataSource(df)

columns = [
        TableColumn(field="clusterNo", title="Cluster"),
        TableColumn(field="title", title="Title"),
        TableColumn(field="author", title="Autoren"),
        TableColumn(field="issued", title="Veröffentlichungsjahr"),
    ]
data_table = DataTable(source=s1, columns=columns, width=640, height=800)

#p.add_widget(widgetbox(data_table))

#show(p)


#s1.callback = 
jscode = CustomJS(args=dict(s2=s2), code="""
    var inds = cb_obj.get('selected')['1d'].indices;
    var d1 = cb_obj.get('data');
    var d2 = s2.get('data');
    
    
    d2["title] = []
    d2["author] = []
    d2["issued] = []
    for (i = 0; i < inds.length; i++) {
        d2["title"].push(d1["title"][inds[i]])
        d2["author"].push(d1["author"][inds[i]])
        d2["issued"].push(d1["issued"][inds[i]])
    }
    s2.trigger('change');
    
""")

p.add_tools(TapTool(callback=OpenURL(url="http://doi.org/@doi")))

# use the "color" column of the CDS to complete the URL
# e.g. if the glyph at index 10 is selected, then @color
# will be replaced with source.data['color'][10]
#url = "http://doi.org/@doi"
#taptool = p.select(type=TapTool)
##taptool = TapTool()
#taptool.callback = OpenURL(url=url)
##p.add_tools(taptool)

layout = row(p, widgetbox(data_table))
#layout = column(p, widgetbox(data_table))

show(layout)
#save(layout)

2020-03-18 16:16:25,844 : INFO : Session output file 'test.html' already exists, will be overwritten.


In [204]:
break

SyntaxError: 'break' outside loop (<ipython-input-204-6aaf1f276005>, line 4)

In [ ]:
df.sample(1)

In [ ]:
vars(s1)
#s1.data

In [206]:

label_source
#type(label_source)

ColumnDataSource(id='1942', ...)

In [207]:
columns = [
        TableColumn(field="title", title="Title"),
        TableColumn(field="author", title="Autoren"),
        TableColumn(field="issued", title="Jahr", formatter=DateFormatter()),
    ]
data_table = DataTable(source=ColumnDataSource(df), columns=columns, width=400, height=280)

show(widgetbox(data_table))

In [214]:
#df_all = pd.DataFrame(index=set(k for k in work_dict.keys() if k in df), columns=["doi", "clusterID", "issued", "title", "authors", "containerTitles", "refCount", "link", "inPlot"])
#for i, doi in enumerate(work_dict.keys()):

#df_dois = set(dois_to_plot & work_dict.keys())
df_dois = set(reference_vectors.vocab.keys() & work_dict.keys())

df_all = pd.DataFrame(index=df_dois, columns=["doi", "issued", "issuedYear", "title", "authors", "containerTitles", "refCount", "refCountNormalized", "link", "clusterID", "clusterKWs", "inPlot"])
for i, doi in enumerate(df_dois):
    
    if i % 1000 == 0:
        print(i, len(df_dois))
    
    pub = work_dict[doi]
    #print(pub)
    df_all.at[doi, "doi"] = doi
    
    
    
    
    try:
        df_all.at[doi, "title"] = pub["title"][0]
    except IndexError:
        pass
        #df_all.at[doi, "title"] = None
    except KeyError:
        pass
    df_all.at[doi, "issued"] = "-".join([str(x) for x in pub["issued"]["date-parts"][0]])
    df_all.at[doi, "issuedYear"] = pub["issued"]["date-parts"][0][0]
    df_all.at[doi, "refCount"] = ref_counts[doi]
    try:
        pub_authors = [a["family"] for a in pub["author"]]
        df_all.at[doi, "authors"] = ", ".join(pub_authors)
    except KeyError:
        df_all.at[doi, "authors"] = "unavailable"
    try:
        df_all.at[doi, "containerTitles"] = pub["container-title"]
    except KeyError:
        pass
    try:
        df_all.at[doi, "link"] = "=Hyperlink(\"https://doi.org/{}\")".format(doi)
    except KeyError:
        pass
    
    if doi in dois_to_plot:
        df_all.at[doi, "inPlot"] = True
        clid = df.at[doi, "clusterNo"]
        df_all.at[doi, "clusterID"] = clid
        if clid in clDict:
            df_all.at[doi, "clusterKWs"] = ", ".join([s for s, totalFreq, clFreq, v in clDict[clid]["vec"] if v>=0.1 and totalFreq>=3])
    else:
        df_all.at[doi, "inPlot"] = False
        
    try:
        year = pub["issued"]["date-parts"][0][0]
        x = ref_counts[doi]
        if year in lambda_mle_ref_count:
            l = lambda_mle_ref_count[year]
        else:
            l = lambda_mle_ref_count_default
        #df_all.at[doi, "expRefCountPercentile"] = 1. - np.exp(-l*x)
        df_all.at[doi, "refCountNormalized"] = l*x
    except KeyError:
        pass

0 31261
1000 31261
2000 31261
3000 31261
4000 31261
5000 31261
6000 31261
7000 31261
8000 31261
9000 31261
10000 31261
11000 31261
12000 31261
13000 31261
14000 31261
15000 31261
16000 31261
17000 31261
18000 31261
19000 31261
20000 31261
21000 31261
22000 31261
23000 31261
24000 31261
25000 31261
26000 31261
27000 31261
28000 31261
29000 31261
30000 31261
31000 31261


In [215]:
df_all = df_all.sort_values(by="refCount", ascending=False)
df_all.sample(10)
df_all.to_excel("df_all.xlsx")

In [216]:
df_all_inPlot = df_all[df_all["inPlot"]==True]
len(df_all_inPlot)

2303

In [217]:
df_all_inPlot.to_excel("df_all_inPlot.xlsx")

In [218]:
df_all.sample(10)

,doi,issued,issuedYear,title,authors,containerTitles,refCount,refCountNormalized,link,clusterID,clusterKWs,inPlot
10.1086/155591,10.1086/155591,1977-10,1977,The size distribution of interstellar grains,"Mathis, Rumpl, Nordsieck",[The Astrophysical Journal],10,0.96164,"=Hyperlink(""https://doi.org/10.1086/155591"")",NaN,NaN,False
10.1080/00336297.2016.1180308,10.1080/00336297.2016.1180308,2016-6-9,2016,Social Justice and the Future of Higher Educat...,Culp,[Quest],4,1.63233,"=Hyperlink(""https://doi.org/10.1080/00336297.2...",NaN,NaN,False
10.1177/1075547010378658,10.1177/1075547010378658,2010-8-18,2010,Gender Differences in Scientists’ Public Outre...,Crettaz von Roten,[Science Communication],5,1.25598,"=Hyperlink(""https://doi.org/10.1177/1075547010...",NaN,NaN,False
10.1063/1.445869,10.1063/1.445869,1983-7-15,1983,Comparison of simple potential functions for s...,"Jorgensen, Chandrasekhar, Madura, Impey, Klein",[The Journal of Chemical Physics],56,7.32954,"=Hyperlink(""https://doi.org/10.1063/1.445869"")",NaN,NaN,False
10.1007/s10648-013-9223-5,10.1007/s10648-013-9223-5,2013-4-12,2013,On Going (Way) Beyond One’s Data: A Proposal t...,"Robinson, Levin, Schraw, Patall, Hunt",[Educational Psychology Review],10,3.00607,"=Hyperlink(""https://doi.org/10.1007/s10648-013...",NaN,NaN,False
10.5194/acp-7-6131-2007,10.5194/acp-7-6131-2007,2007-12-14,2007,Closure study between chemical composition and...,"Gysel, Crosier, Topping, Whitehead, Bower, Cub...",[Atmospheric Chemistry and Physics],21,3.99416,"=Hyperlink(""https://doi.org/10.5194/acp-7-6131...",NaN,NaN,False
10.3390/s111009887,10.3390/s111009887,2011-10-20,2011,Cooperative MIMO Communication at Wireless Sen...,"Islam, Han",[Sensors],4,1.08477,"=Hyperlink(""https://doi.org/10.3390/s111009887"")",NaN,NaN,False
10.1086/652373,10.1086/652373,2010-6,2010,Conceptual Synthesis in Community Ecology,Vellend,[The Quarterly Review of Biology],17,4.27033,"=Hyperlink(""https://doi.org/10.1086/652373"")",NaN,NaN,False
10.1021/nl500515q,10.1021/nl500515q,2014-5-27,2014,Tuning Interlayer Coupling in Large-Area Heter...,"Tongay, Fan, Kang, Park, Koldemir, Suh, Narang...",[Nano Letters],6,1.87038,"=Hyperlink(""https://doi.org/10.1021/nl500515q"")",NaN,NaN,False
10.1006/mssp.2002.1489,10.1006/mssp.2002.1489,2002-7,2002,BLADE-TIP TIMING MEASUREMENT OF SYNCHRONOUS VI...,"DIMITRIADIS, CARRINGTON, WRIGHT, COOPER",[Mechanical Systems and Signal Processing],10,1.67915,"=Hyperlink(""https://doi.org/10.1006/mssp.2002....",NaN,NaN,False


In [213]:
dois_to_plot.index("10.1021/op200097d")

ValueError: '10.1021/op200097d' is not in list

Idee:

für Bestimmung der Wahrscheinlichkeit für Referenz eines Papers bei einer vorhandenen Menge an bekannten Referenzen

Kreuzvalidierung möglicher Metriken mit Subset der Referenz-Relationen